#🤖 Fine-Tuning T5 for Product Review Generation

In this interactive lab, we'll explore the exciting task of generating product reviews using the T5 (Text-to-Text Transfer Transformer) model. We'll dive into data preparation, model training, and ultimately, review generation.

###Setup and Installation

First things first, we need to install the required libraries to ensure our environment is ready for the tasks ahead.

In [ ]:
!pip install numpy==1.25.1
!pip install transformers
!pip install datasets===2.13.1

###Importing Libraries 📚

Let's import all the necessary modules that will help us load datasets, process data, and utilize the T5 model.


In [ ]:
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding

###Data Preparation 📋

Our journey begins with preparing our dataset. We'll use a subset of Amazon product reviews for our analysis and training.

Loading and Merging Datasets
We replace the unavailable "amazon_us_reviews" with a similar dataset and merge metadata with review data.

In [ ]:
# Amazon removed the "amazon_us_reviews" dataset, so we'll have to use a replacement here.
dataset_category = "Software" # "Electronics" you can also choose electronics like in the lesson, but the dataset is bigger and loading will take longer

meta_ds = load_dataset("McAuley-Lab/Amazon-Reviews-2023", f"raw_meta_{dataset_category}", split='full').to_pandas()[['parent_asin', 'title']]
review_ds = load_dataset("McAuley-Lab/Amazon-Reviews-2023", f"raw_review_{dataset_category}", split='full').to_pandas()[['parent_asin', 'rating', 'text', 'verified_purchase']]

ds = meta_ds.merge(review_ds, on='parent_asin', how='inner').drop(columns="parent_asin")
ds = ds.rename(columns={"rating":"star_rating", "title":"product_title", "text":"review_body"})

ds = ds[ds['verified_purchase'] & (ds['review_body'].map(len) > 100)].sample(100_000)
ds

Encoding and Splitting
Next, we encode our star_rating column and split our dataset into training and testing sets.

In [ ]:
# Loading the dataset
dataset = Dataset.from_pandas(ds)

# encoding the 'star_rating' column
dataset = dataset.class_encode_column("star_rating")

# Splitting the dataset into training and testing sets
dataset = dataset.train_test_split(test_size=0.1, seed=42, stratify_by_column="star_rating")

train_dataset = dataset['train']
test_dataset = dataset['test']
print(train_dataset[0])

###Model Preparation 🛠️

Now, let's prepare our T5 model for training.

###Tokenizer Initialization

In [ ]:
MODEL_NAME = 't5-base'
tokenizer = T5Tokenizer.from_pretrained('t5-base')|

###Data Preprocessing Function
We define a function to preprocess our data, preparing it for the model.

In [ ]:
# Defining the function to preprocess the data
def preprocess_data(examples):
    examples['prompt'] = [f"review: {example['product_title']}, {example['star_rating']} Stars!" for example in examples]
    examples['response'] = [f"{example['review_headline']} {example['review_body']}" for example in examples]

    inputs = tokenizer(examples['prompt'], padding='max_length', truncation=True, max_length=128)
    targets = tokenizer(examples['response'], padding='max_length', truncation=True, max_length=128)

    # Set -100 at the padding positions of target tokens
    target_input_ids = []
    for ids in targets['input_ids']:
        target_input_ids.append([id if id != tokenizer.pad_token_id else -100 for id in ids])

    inputs.update({'labels': target_input_ids})
    return inputs

###Preprocessing Datasets

In [ ]:
train_dataset = train_dataset.map(preprocess_data, batched=True)
test_dataset = test_dataset.map(preprocess_data, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

###Fine-Tuning the Model 🎯

With our data ready, we proceed to fine-tune the T5 model on our dataset.

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

TRAINING_OUTPUT = "./models/t5_fine_tuned_reviews"
training_args = TrainingArguments(
    output_dir=TRAINING_OUTPUT,
    num_train_epochs=3,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    save_strategy='epoch',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

trainer.train()

###Saving and Loading the Model 💾

After training, we save our model for later use and demonstrate how to load it.

In [ ]:
trainer.save_model(TRAINING_OUTPUT)

In [ ]:
# Loading the fine-tuned model
model = T5ForConditionalGeneration.from_pretrained(TRAINING_OUTPUT)

# or get it directly trained from here:
# model = T5ForConditionalGeneration.from_pretrained("TheFuzzyScientist/T5-base_Amazon-product-reviews")

###Generating Reviews ✍️

Finally, we use our fine-tuned model to generate reviews for new products.

In [ ]:
# Defining the function to generate reviews
def generate_review(text):
    inputs = tokenizer("review: " + text, return_tensors='pt', max_length=512, padding='max_length', truncation=True)
    outputs = model.generate(inputs['input_ids'], max_length=128, no_repeat_ngram_size=3, num_beams=6, early_stopping=True)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

In [ ]:
# Generating reviews for random products
random_products = test_dataset.shuffle(42).select(range(10))['product_title']

print(generate_review(random_products[0] + ", 3 Stars!"))
print(generate_review(random_products[1] + ", 5 Stars!"))
print(generate_review(random_products[2] + ", 2 Stars!"))

### Conclusion 🚀

Congratulations! You've just completed a hands-on project on fine-tuning T5 for generating product reviews. Experiment further with different product categories or tweak the model parameters to see how it affects the output. Happy coding!

## 🚀 Next Steps: Elevate Your Skills with Advanced Techniques

Congratulations on reaching this far! If you're keen to expand your expertise and dive deeper into the world of Large Language Models (LLMs), our next course is designed just for you.

### 🌟 Use LLMs Smarter: Scale Gen AI, ML-Ops & Cost Efficiency
In a world where AI and machine learning are revolutionizing industries, the ability to deploy and manage massive models like Llama, Mistral, and Gemma efficiently is invaluable.

This course is tailored to equip you with the knowledge and skills to:

* **Deploy Huge Models**: Learn the ins and outs of working with some of the largest models available, understanding their architecture and how they can be leveraged for your projects.
* **Scale Across Clusters**: Discover strategies for scaling these behemoths across clusters of machines without sacrificing performance, ensuring seamless operation.
* **Optimize Response Times**: Achieve response times in the milliseconds while maintaining the delicate balance between accuracy and speed.
* **Balance Accuracy, Speed, & Cost**: Master the art of cost efficiency without compromising on performance, utilizing the latest and most powerful technologies.

### 🎓 Why This Course?
This course goes beyond the basics, diving into the practical aspects of deploying and optimizing LLMs at scale. Whether you're working on cutting-edge research or developing solutions for real-world problems, the insights and techniques covered here will be invaluable.

### 💡 Take the Leap
If you're intrigued by the possibilities and ready to take your skills to the next level, take 2 minutes to explore this course further. As a token of our gratitude for completing the current course, we're offering an exclusive discount—better than what you might find elsewhere—for this next step in your journey.

# 🔗 Check out the course [here](https://www.udemy.com/course/deploy-ai-smarter-llm-scalability-ml-ops-cost-efficiency/?referralCode=ADC24A974EEC326467E6/?couponCode=D0627D21CFA8214F8939)

Use this cuppon code if the discount is not applied automatically: **D0627D21CFA8214F8939**


Embrace the opportunity to become a proficient practitioner in deploying, scaling, and optimizing Large Language Models. Your journey into the advanced realms of AI and machine learning starts now!